In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math

from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



try:
    import cPickle as pickle  ##################################
   #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
#import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
import time  


import sys

import plotly.plotly as py
import plotly.graph_objs as go

#i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
import plotly.tools as tls
tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


#import pygraphviz
#from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



# ##### for getting geolocation data  and to calculate distance between two geolocations
# import requests
# import json
#import geopy.distance   


In [2]:

# ######## load dictionaries for user: list_folders, and folder: list_users
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_list_folders_COMPLETE.pickle'
with open(pickle_name, 'rb') as handle:
    dict_user_id_list_folders = pickle.load(handle)
print "num. users:",len(dict_user_id_list_folders)




pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_list_users_COMPLETE.pickle'
with open(pickle_name, 'rb') as handle:
    dict_folder_id_list_users = pickle.load(handle)
print "num. folders:",len(dict_folder_id_list_users)
#######################################################





# # #######   dict user: number tot act
# # pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_tot_act_PARTIAL.pickle'
# # with open(pickle_name, 'rb') as handle:
# #     dict_user_id_tot_act = pickle.load(handle)
# # print "num. users:",len(dict_user_id_tot_act)
# # #####################################################




########  csv  users basic attr
path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file = 'new_DROPBOX_basic_user_attr.csv'
df_user_basic_attr = pd.read_csv(path+input_file, sep=';',na_values=["NAN","-1","null"]) # set header=0 if i wanna pass it my own list of header names
df_user_basic_attr.drop('Unnamed: 0', axis=1, inplace=True)
print "num. users:",df_user_basic_attr.shape





#########  csv  all act aggr by date
path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file = 'DROPBOX_only_activity_aggr_by_date_COMPLETE.csv'
%time df_all_act = pd.read_csv(path+input_file, sep=',',na_values=["NAN","NaN","nan","-1","null",""], parse_dates=['folder_creation_date','date']) 
df_all_act.drop('Unnamed: 0', axis=1, inplace=True)
print "num. users:",len(df_all_act.user_id.unique()),df_all_act.shape    # 400000   (5992006, 7)
                   
#df_all_act=df_all_act[df_all_act['date'] != "nan"]  # for some reason, the nans, are interpreted as str  :()   #df_all_act[df_all_act.date.notnull()]    


df_all_act=df_all_act.replace('nan', np.nan)  ## for some reason, there are diff nomenclatures for NANs, and it is not interpreting them correctly when reading the file 
df_all_act=df_all_act.replace('NAN', np.nan)
df_all_act=df_all_act.replace('NaN', np.nan)
df_all_act=df_all_act.replace('-1', np.nan)
df_all_act=df_all_act.replace('', np.nan)


df_all_act= df_all_act[df_all_act.date.notnull()]    
print "after removing rows without dates:\nnum. users:",len(df_all_act.user_id.unique()),df_all_act.shape    # 400000  (5141758, 7)


num. users: 438284
num. folders: 519045
num. users: (438300, 15)
num. users: 400000 (5992006, 7)


In [60]:

def get_alternation(df_select, folder_id, list_users):  # df_select is for a given folder_id, sorted by date and excluding entries without a date (== for users without actual act)
    
#     print folder_id
#     print list_users
#     print df_select
    
    cont_alternating_person=0.
    old_person=0
    list_users_in_order_of_activity=df_select.user_id.values
    #print list_users_in_order_of_activity
    if len(set(list_users_in_order_of_activity))>1:  # more than one user
        for i in range(len(list_users_in_order_of_activity)) :

            person=list_users_in_order_of_activity[i]
            if i >0:
                if person != old_person:
                    cont_alternating_person +=1.
            old_person = person            

    
        norm=float(len(list_users_in_order_of_activity)-1.)
        alt=cont_alternating_person/norm
       # print "cont_alt:",cont_alternating_person, "  num_events-1:",norm, "  alt:",alt   
    else:
        alt=np.nan
      # print "  alt:",alt, " only one user"   
    
   
    return alt
    
    
#############################################
#############################################




dict_folder_id_alternation={}

for folder_id in tqdm_notebook(dict_folder_id_list_users.keys()):
    
    if len(dict_folder_id_list_users[folder_id])>1: # two or more members:
        df_select=df_all_act[df_all_act['folder_id'] == folder_id].sort_values(by=['date'])
    
        if len(df_select) >1:  # more than one record
            list_users=dict_folder_id_list_users[folder_id]
        
            dict_folder_id_alternation[folder_id]=get_alternation(df_select, folder_id, list_users)    
    
        else:
            dict_folder_id_alternation[folder_id]=np.nan
    else:
        dict_folder_id_alternation[folder_id]=np.nan
       

    
    
    
print "done"    
        
###########################################

pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_alternation_PARTIAL.pickle'
with open(pickle_name, 'wb') as handle:
    pickle.dump(dict_folder_id_alternation, handle)
print "written:", pickle_name
    
    

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/juliaponcela/anaconda2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/juliaponcela/anaconda2/lib/python2.7/site-packages/tqdm/_tqdm.py", line 103, in run
    for instance in self.tqdm_cls._instances:
  File "/home/juliaponcela/anaconda2/lib/python2.7/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



done
written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_alternation_PARTIAL.pickle


In [132]:
# for folder_id in dict_folder_id_alternation:
#     print folder_id, dict_folder_id_alternation[folder_id]

In [131]:
# df_all_act.folder_id.unique()
# # user_id=340700730.0
# # df_select=df_all_act[df_all_act['user_id'] == user_id]
# # df_select
# list_folders_more2=[]
# for folder_id in df_all_act.folder_id.unique():
#     lista=list(df_all_act[df_all_act['folder_id'] == folder_id].user_id.unique())
    
#     if len(lista ) > 3:
#         list_folders_more2.append(folder_id)
#         #print item